In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ipykernel

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile app.py

import os
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import ImageDraw
from PIL import Image
import pycocotools
import torch
import torch.utils.data
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import streamlit as st

# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T
import torchvision
from torchvision import transforms

def get_model(num_classes):
   model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
   in_features = model.roi_heads.box_predictor.cls_score.in_features
   model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
   return model

def getlabelName(num):
  m= list(csv2[csv2["label"] == num]['class'].values)
  return m[0]


csv2= pd.read_csv('/content/drive/MyDrive/bottomfinalcoordinates_1.csv')
weights= '/content/drive/MyDrive/detection_model_bottoms1gb' 
#bottom 17, top 18
loaded_model = get_model(num_classes = 17)
loaded_model.load_state_dict(torch.load(weights))

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("LEVIS DEFECT DETECTION")
st.write("")

file_up = st.file_uploader("Upload an image", type="jpg")
if file_up is not None:
  img = Image.open(file_up)
  img1= img
  st.image(img1, caption='Uploaded Image.', use_column_width=True)
  st.write("")
  draw = ImageDraw.Draw(img1)

  transform = transforms.Compose([transforms.ToTensor()])
  batch_t = torch.unsqueeze(transform(img),0)
  img = batch_t[0]
  loaded_model.eval()
  with torch.no_grad():
    prediction = loaded_model([img])
  st.write("Just a second...")
  count =0
  for element in range(len(prediction[0]["boxes"])):
    if count <3:
      colors = ['red','blue','green']
      boxes = prediction[0]["boxes"][element].cpu().numpy()
      score = np.round(prediction[0]["scores"][element].cpu().numpy(),decimals= 4)
      label = prediction[0]["labels"][element]
      # st.write(score) 
      draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline =colors[count], width =3)
      draw.text((boxes[0], boxes[1]), text = str(score))
      lb = getlabelName(label.numpy())
      draw.text((boxes[0], boxes[3]), text = str(lb))
      st.write(colors[count]+ " = "+ str(lb) + " score : " + str(score))
      count = count+1

  st.image(img1, caption='Uploaded Image.', use_column_width=True)

Writing app.py


In [ ]:
#install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-05-05 06:44:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.235.211.105, 34.193.233.154, 52.204.190.140, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.235.211.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  78.9MB/s    in 0.2s    

2021-05-05 06:44:05 (78.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://48720a345965.ngrok.io


In [ ]:
!streamlit run /content/app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.199.23.186:8501

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100% 160M/160M [00:00<00:00, 209MB/s]
  Stopping...


In [ ]:
x=5
y = [4,5,3]
if x in y:
  print(x)

5
